Code to calculate for each granule which orbit has the maximum coverage by using the count of the valid pixel layer

In [1]:
import os
import pip
import virtualenv

# Activate the venv
venv_dir = "/home/users/abdb2/nb-venvs/venv-notebook"
activate_file = os.path.join(venv_dir, "bin", "activate_this.py")
exec(open(activate_file).read(), dict(__file__=activate_file))
print ("Activating venv complete")

Activating venv complete


In [ ]:
import os
import sys
import glob
import gdal
import numpy as np
import rasterio
import pandas as pd

# Get pixel mask filenames 
wd = '/neodc/sentinel_ard/data/sentinel_2/2020/11'
grantoproc = ['T29UPB', 'T29VND', 'T29VNE', 'T29VPC', 'T29VPD', 'T29VPE', 'T30UUF', 'T30UUG', 
                 'T30UVF', 'T30UVG', 'T30UWG', 'T30VUH', 'T30VUJ', 'T30VUK', 'T30VUL', 'T30VVH', 
                 'T30VVJ', 'T30VVK', 'T30VVL', 'T30VWH', 'T30VWJ', 'T30VWL', 'T30VWM', 'T30VWN', 'T30VXM', 'T30VXN']

df = pd.DataFrame(columns = ['Granule','Orbit','Coverage'])
row = 0

# r = root, d = directories, f = filenames
for r, d, f in os.walk(wd, followlinks=True):
    for name in glob.fnmatch.filter(f, '*osgb_valid.tif'): # exclude Irish projection
        if name.split('_')[3] in grantoproc:
            with rasterio.open(os.path.join(r, name), 'r') as s2_image:
                valid_array = s2_image.read(1).astype('uint16')
                sum_pixel = np.sum(valid_array)
            df.loc[row] = [name.split('_')[3],name.split('_')[4],sum_pixel]
            row = row+1
            
# sort and print dataframe
df = df.sort_values(["Granule", "Orbit", "Coverage"], ascending = (True, True, True))
#print(df)

# get max value per granule/orbit combination
df_max = df.groupby(['Granule', 'Orbit']).agg({'Coverage': ['max']})
df_max.columns = ['pixel_max']
df_max = df_max.reset_index()
#print(df_max)

# get max orbit number
idx = df_max.groupby(['Granule'])['pixel_max'].transform(max) == df_max['pixel_max']
df_max_orbit = df_max[idx]
df_max_orbit = df_max_orbit.reset_index()
print(df_max_orbit)

Test dictionaries

In [45]:
orbit_dict = {'T29UPB':['ORB123'],
              'T29VND':['ORB023'],
              'T29VNE':['ORB023'],
              'T29VPC':['ORB123'],
              'T29VPD':['ORB123'],
              'T29VPE':['ORB023'],
              'T30UUF':['ORB080'],
              'T30UUG':['ORB123'],
              'T30UVF':['ORB080'],
              'T30UVG':['ORB080'],
              'T30UWG':['ORB037'],
              'T30VUH':['ORB123'],
              'T30VUJ':['ORB123'],
              'T30VUK':['ORB123'],
              'T30VUL':['ORB023', 'ORB123'],
              'T30VVH':['ORB080'],
              'T30VVJ':['ORB080'],
              'T30VVK':['ORB123'],
              'T30VVL':['ORB123'],
              'T30VWH':['ORB037', 'ORB080'],
              'T30VWJ':['ORB080'],
              'T30VWL':['ORB080'],
              'T30VWM':['ORB080'],
              'T30VWN':['ORB080','ORB123'],
              'T30VXM':['ORB080'],
              'T30VXN':['ORB080'], 
               }

gran = 'T30VWQ'
orbit_list = orbit_dict.get(gran, 'granule not in dictionary')
print(orbit_list)

orb = 'ORB080'

if orb in orbit_list:
    print('match found')
else:
    print("match_not_found")


granule not in dictionary
match_not_found
